In [ ]:
import os
import glob
from google.colab import drive
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# --- User Configuration ---
# Specify the path to your dataset ZIP file in Google Drive
dataset_zip_path = "/content/drive/MyDrive/YOLOv5-Thermal.v1i.yolov5pytorch.zip"
# --------------------------

# 1. Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# Check if the dataset file exists
if not os.path.exists(dataset_zip_path):
  print(f"ERROR: Dataset ZIP file not found at: {dataset_zip_path}")
  print("Please update the 'dataset_zip_path' variable with the correct path.")
else:
  # 2. Copy the dataset to Colab runtime and unzip
  print("Copying and unzipping dataset...")
  # Extract the base name of the zip file to use it in the colab runtime
  zip_file_name = os.path.basename(dataset_zip_path)
  !cp "{dataset_zip_path}" /content/
  !unzip -q /content/"{zip_file_name}" -d /content/dataset
  print("Dataset successfully unzipped into /content/dataset")

  # 3. Clone YOLOv5 repository and install dependencies
  print("Cloning YOLOv5 repository...")
  !git clone https://github.com/ultralytics/yolov5 /content/yolov5
  %cd /content/yolov5

  print("Installing dependencies...")
  !pip install -qr requirements.txt
  print("Dependencies installed.")

  # 4. Train the YOLOv5 model
  print("Starting YOLOv5 training...")
  # Using smaller batch size and fewer epochs for faster training, and yolov5n.pt for a smaller model.
  # Ensure GPU runtime is selected in Colab (Runtime -> Change runtime type -> T4 GPU)
  !python train.py --img 640 --batch 16 --epochs 25 --data /content/dataset/data.yaml --weights yolov5n.pt --device 0 # --device 0 ensures GPU is used
  print("Training finished.")

  # 5. Run inference on test images
  print("Running inference on test images...")
  # The best trained weights are saved in the 'runs/train/exp*/weights/best.pt'
  # We use the latest experiment folder found by glob
  latest_exp_folder = sorted(glob.glob('/content/yolov5/runs/train/exp*'))[-1]
  best_weights_path = os.path.join(latest_exp_folder, 'weights/best.pt')

  # Define the source for test images, assuming a standard dataset structure
  test_images_path = '/content/dataset/test/images'
  if not os.path.exists(test_images_path):
    # Fallback for different structures, e.g., 'valid' or 'validation'
    if os.path.exists('/content/dataset/valid/images'):
      test_images_path = '/content/dataset/valid/images'
    elif os.path.exists('/content/dataset/validation/images'):
      test_images_path = '/content/dataset/validation/images'
    else:
      print(f"Could not find test images directory at {test_images_path} or common fallbacks.")
      test_images_path = None

if test_images_path:
  !python detect.py --weights "{best_weights_path}" --img 640 --conf 0.25 --source "{test_images_path}" --device 0 # --device 0 ensures GPU is used
  print("Inference complete.")

  # 6. Display a prediction result
  print("Displaying a sample prediction...")
  # Find the latest detection folder
  latest_detect_folder = sorted(glob.glob('/content/yolov5/runs/detect/exp*'))[-1]

  # Get a list of result images
  result_images = glob.glob(os.path.join(latest_detect_folder, '*.jpg'))

  if result_images:
      # Select the first image to display
      img_path = result_images[0]

      # Display the image
      plt.figure(figsize=(12, 12))
      img = mpimg.imread(img_path)
      plt.imshow(img)
      plt.axis('off') # Hide axes
      plt.title(f"Displaying result: {os.path.basename(img_path)}")
      plt.show()
  else:
      print("No prediction images found to display.")
else:
   print("Skipping inference as test image directory was not found.")

Mounting Google Drive...
Mounted at /content/drive
Copying and unzipping dataset...
Dataset successfully unzipped into /content/dataset
Cloning YOLOv5 repository...
Cloning into '/content/yolov5'...
remote: Enumerating objects: 17511, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 17511 (delta 5), reused 0 (delta 0), pack-reused 17491 (from 3)
Receiving objects: 100% (17511/17511), 16.60 MiB | 28.62 MiB/s, done.
Resolving deltas: 100% (11993/11993), done.
/content/yolov5
Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from google.colab import drive
import glob

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the unzipped dataset
dataset_path = '/content/dataset'
print(f"Dataset path: {dataset_path}")

# Define the path to the data.yaml file within the dataset
data_yaml_path = os.path.join(dataset_path, 'data.yaml')
print(f"Data YAML path: {data_yaml_path}")

# Update the data.yaml file to use absolute paths
if os.path.exists(data_yaml_path):
    with open(data_yaml_path, 'r') as f:
        data_yaml_content = f.read()

    # Replace relative paths with absolute paths
    data_yaml_content = data_yaml_content.replace('train: ../train/images', f'train: {dataset_path}/train/images')
    data_yaml_content = data_yaml_content.replace('val: ../valid/images', f'val: {dataset_path}/valid/images')

    with open(data_yaml_path, 'w') as f:
        f.write(data_yaml_content)
    print("data.yaml updated with absolute paths.")
else:
    print(f"ERROR: data.yaml not found at: {data_yaml_path}")


# Train the model
# Change to the yolov5 directory before training
%cd /content/yolov5
!yolo task=detect mode=train model=yolov8n.pt data={data_yaml_path} epochs=50 imgsz=640 batch=32

# Define the path to the trained model
# The path is relative to the yolov5 directory
trained_model_path = 'runs/detect/train/weights/best.pt'
full_trained_model_path = os.path.join('/content/yolov5', trained_model_path)
print(f"Expected trained model path: {full_trained_model_path}")


# Define the destination path in Google Drive
drive_destination_path = '/content/drive/My Drive/trained_yolov8_model.pt'
print(f"Google Drive destination path for model: {drive_destination_path}")

# Copy the trained model to Google Drive
# Execute the copy command from the yolov5 directory
!cp {trained_model_path} "{drive_destination_path}"


# Define the source directory for test images
test_images_path = '/content/dataset/test/images'
if not os.path.exists(test_images_path):
  # Fallback for different structures, e.g., 'valid' or 'validation'
  if os.path.exists('/content/dataset/valid/images'):
    test_images_path = '/content/dataset/valid/images'
  elif os.path.exists('/content/dataset/validation/images'):
    test_images_path = '/content/dataset/validation/images'
  else:
    print(f"Could not find test images directory at {test_images_path} or common fallbacks.")
    test_images_path = None

if test_images_path:
  print(f"Test images path: {test_images_path}")
  # Perform inference on test images
  # The path to the trained model needs to be the full path now
  full_trained_model_path = os.path.join('/content/yolov5', trained_model_path)
  !yolo task=detect mode=predict model={full_trained_model_path} source="{test_images_path}"

  # Define the path to the inference results images
  # Find the latest detection folder
  latest_detect_folder = sorted(glob.glob('/content/yolov5/runs/detect/predict*'))[-1]
  inference_images_path = latest_detect_folder
  print(f"Inference results images path: {inference_images_path}")


  # Define the path for the generated video in the Colab workspace
  generated_video_path = '/content/inference_results_video.mp4'
  print(f"Generated video path in Colab workspace: {generated_video_path}")

  # Generate video from inference results images using FFmpeg
  # Ensure FFmpeg is installed (usually pre-installed in Colab)
  # The -r flag sets the frame rate (e.g., 10 frames per second)
  # The -s flag sets the resolution (use the image size)
  # The -pix_fmt yuv420p is for compatibility
  # The *.jpg pattern assumes your images are in JPG format. Adjust if necessary.
  !ffmpeg -y -r 10 -i "{inference_images_path}/%01d.jpg" -vcodec libx264 -pix_fmt yuv420p {generated_video_path}

  # Define the destination path for the generated video in Google Drive
  generated_video_drive_path = '/content/drive/My Drive/inference_results_video.mp4'
  print(f"Google Drive destination path for video: {generated_video_drive_path}")

  # Copy the inference results video to Google Drive
  # Execute the copy command from the yolov5 directory
  # We need to change back to the root directory to copy from /content
  %cd /content/
  !cp {generated_video_path} "{generated_video_drive_path}"
  %cd /content/yolov5 # Change back to yolov5 directory if needed for subsequent steps

  # Display the generated video (optional, for verification)
  # from IPython.display import Video
  # Video(generated_video_drive_path)
else:
   print("Skipping inference and video generation as test image directory was not found.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset path: /content/dataset
Data YAML path: /content/dataset/data.yaml
data.yaml updated with absolute paths.
/content/yolov5
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max